<a href="https://colab.research.google.com/github/cbonnin88/RailFlow/blob/main/Data_Generation_RailFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import random
import uuid
import hashlib
from faker import Faker
from datetime import datetime, timedelta

In [4]:
# Initialize Faker (French Locale for realism)
fake = Faker('fr_FR')
Faker.seed(42)
np.random.seed(42)

In [5]:
# Configuration
NUM_USERS = 5000
NUM_SEARCHES = 15000
NUM_BOOKINGS = 4000 # Not all searches result in bookings

print('Generating RailFlow Dirty Databases')

Generating RailFlow Dirty Databases


# **Generate users (users.csv)**

In [6]:
print(f'Generating {NUM_USERS} users...')

user_ids = [str(uuid.uuid4()) for _ in range(NUM_USERS)]
sub_types = ['Free','Max Jeune','Max Senior', None] # None = Dirty Data

users_data = {
    'user_id': user_ids,
    'signup_date': [fake.date_between(start_date='-2y', end_date='today') for _ in range(NUM_USERS)],
    'subscription_type': [np.random.choice(sub_types, p=[0.6,0.2,0.1,0.1]) for _ in range(NUM_USERS)],
    'age': [fake.random_int(min=18,max=90) for _ in range(NUM_USERS)],
    'device_os': [np.random.choice(['iOS','Android','Web','Unknown']) for _ in range(NUM_USERS)]
}

df_users = pd.DataFrame(users_data)

Generating 5000 users...


# **Injecting Dirt (Users)**

1. Duplicates: Duplicate 5% of users

In [7]:
duplicates = df_users.sample(frac=0.05)
df_users = pd.concat([df_users, duplicates])

In [8]:
# Dirty Ages: Converting some ages to weird strings or negative numbers
def dirty_age(age):
  choice = np.random.rand()
  if choice < 0.02: return f'{age}yo', # String with text
  if choice < 0.04: return -age, # Negative logic error
  if choice < 0.05: return 'unknown' # String text
  return age

df_users['age'] = df_users['age'].apply(dirty_age)

In [9]:
df_users.to_csv('users.csv', index=False)
print('users.csv created')

users.csv created


In [10]:
df_users.head()

,user_id,signup_date,subscription_type,age,device_os
0,36065652-c3de-492b-813f-2d576b61c6c0,2025-05-02,Free,34,Web
1,1e7c8c84-7170-4f62-bd2b-0b9ea8052c15,2024-02-08,None,90,Unknown
2,bda20fbc-06b4-4df9-891c-a4fe37ba21b8,2024-08-09,Max Jeune,81,Web
3,aa47fbea-3da2-44ca-84b0-051b9a4f8c97,2024-07-02,Free,84,Unknown
4,28d4a58b-d921-4705-a2f8-6d8123c14de1,2025-07-12,Free,81,Unknown


# **Generate Searches (searches.csv)**

In [11]:
print(f'Generating {NUM_SEARCHES} searches')

stations = ['Paris Gare de Lyon', 'Lyon Part-Dieu', 'Marseille St-Charles', 'Bordeaux St-Jean', 'Lille Europe', 'Strasbourg', 'Nantes']

search_ids = [str(uuid.uuid4()) for _ in range(NUM_SEARCHES)]
# Pick random users for searches
search_user_ids = [np.random.choice(user_ids) for _ in range(NUM_SEARCHES)]

searches_data = {
    'search_id': search_ids,
    'user_id': search_user_ids,
    'timestamp': [fake.date_time_between(start_date='-1y',end_date='now') for _ in range(NUM_SEARCHES)],
    'origin_station':[np.random.choice(stations) for _ in range(NUM_SEARCHES)],
    'destination_station': [np.random.choice(stations) for _ in range(NUM_SEARCHES)],
    'passenger_count': [np.random.choice([1,2,3,4], p=[0.7,0.2,0.05,0.05]) for _ in range(NUM_SEARCHES)]
}

df_searches = pd.DataFrame(searches_data)

Generating 15000 searches


# **Injecting Dirty Data**

In [12]:
df_searches['departure_date'] = df_searches['timestamp'] + pd.to_timedelta(np.random.randint(0,30,size=NUM_SEARCHES), unit='D')

In [13]:
# 1. Casing issues: 'Paris' vs 'PARIS' vs 'paris'

def dirty_stations(station):
  choice = np.random.rand()
  if choice < 0.10: return station.upper()
  if choice < 0.20: return station.lower()
  if choice < 0.25: return f'{station}' # Trailing whitespace
  return station

df_searches['destination_station'] = df_searches['destination_station'].apply(dirty_stations)

In [14]:
# 2. Time Travel (Logical Error): Departure date BEFORE search date

mask = df_searches.index > (NUM_SEARCHES * 0.98)
df_searches.loc[mask, 'departure_date'] = df_searches.loc[mask,'timestamp'] - pd.to_timedelta(5,unit='D')

In [15]:
# 3. Mixed Timestamp Formats (ISO String vs Datetime Obj)

def dirty_time(ts):
  if np.random.rand() < 0.15:
    return ts.strftime('%Y/%m/%d %H:%M:%S') # wrong separator
  return ts.isoformat()

df_searches['timestamp'] = df_searches['timestamp'].apply(dirty_time)

In [16]:
df_searches.to_csv('searches.csv',index=False)
print('searches.csv created')

searches.csv created


In [ ]:
df_searches.head()

,search_id,user_id,timestamp,origin_station,destination_station,passenger_count,departure_date
0,90927b8a-830e-4eaa-afc2-3c669cd2ef25,7de933dd-3b90-471a-81df-dfa8e0e23645,2025/05/10 13:49:18,Nantes,Strasbourg,1,2025-05-30 13:49:18.419189
1,23007a71-b9d4-4db7-8f6d-08ae836e71f2,885fd832-1e47-4269-811a-7292b790ba9e,2025-01-18T04:42:05.746525,Marseille St-Charles,Nantes,1,2025-01-23 04:42:05.746525
2,f574a5b5-28d2-48b4-9fdd-c0a91b69262a,78a61a2d-d26b-40be-8d8b-d00228ccca37,2025-05-14T07:56:09.490315,Lyon Part-Dieu,Nantes,2,2025-05-21 07:56:09.490315
3,8ad8fa65-a7d1-4206-b344-55eaf5d7efad,d1b483dc-59e4-425e-9bc7-267e124b2e80,2025-07-03T04:31:32.365211,Bordeaux St-Jean,Bordeaux St-Jean,3,2025-08-01 04:31:32.365211
4,02735e43-f2c4-49a3-b36b-8668f8bfdb9b,fe6c2362-70bb-4548-a281-34bfbf9a9b72,2025-10-31T16:01:25.469735,Lille Europe,Paris Gare de Lyon,1,2025-11-05 16:01:25.469735


# **Generate Bookings**

In [17]:
print(f'Generating {NUM_BOOKINGS} bookings')

# Randomly select searches that converted into bookings
converted_searches = np.random.choice(search_ids, size=NUM_BOOKINGS, replace=False)

bookings_data = {
    'booking_id': [str(uuid.uuid4()) for _ in range(NUM_BOOKINGS)],
    'search_id': converted_searches,
    'payment_status': [np.random.choice(['Success', 'Failed', 'Pending'], p=[0.85, 0.1, 0.05]) for _ in range(NUM_BOOKINGS)],
    'ticket_class': [np.random.choice(['1st Class', '2nd Class'], p=[0.2, 0.8]) for _ in range(NUM_BOOKINGS)],
    'amount_eur': np.round(np.random.uniform(25, 150, size=NUM_BOOKINGS), 2)
}

df_bookings = pd.DataFrame(bookings_data)

Generating 4000 bookings


# **Injecting Dirt (Bookings)**

In [18]:
# 1. Currency Symbols (Type Mismatch): Mix floats and strings like "€45.50"
def dirty_price(price):
    if np.random.rand() < 0.2:
        return f"€{price}"
    return price

df_bookings['amount_eur'] = df_bookings['amount_eur'].apply(dirty_price)

In [19]:
# 2. Orphan Bookings: Create bookings with a search_id that DOES NOT exist in searches.csv
# We generate 50 random orphan bookings
orphans = pd.DataFrame({
    'booking_id': [str(uuid.uuid4()) for _ in range(50)],
    'search_id': [str(uuid.uuid4()) for _ in range(50)], # These UUIDs won't match searches
    'payment_status': 'Success',
    'ticket_class': '2nd Class',
    'amount_eur': 50.0
})

df_bookings = pd.concat([df_bookings, orphans])

In [20]:
df_bookings.to_csv('bookings.csv', index=False)
print('bookings.csv created')

bookings.csv created


In [21]:
df_bookings.head()

,booking_id,search_id,payment_status,ticket_class,amount_eur
0,d790638e-b87b-4a8f-8318-4cd7ab939c10,a3c05efe-c5fc-4656-87ae-490be2f6b6a8,Success,2nd Class,71.81
1,5459534a-0f46-4450-bc2e-49f3f29558e8,0deaf598-2144-45f1-908e-3bfa72799af2,Success,2nd Class,140.79
2,0cf39d8f-e88b-4ab6-ba3b-4fa282f3e06b,6e568ccf-af1c-4141-b276-ddc86b9f67f6,Success,2nd Class,32.69
3,d0677be4-d639-4b95-a364-793f66473372,121ef2fb-8eb6-4272-b8ea-9a7537f516a8,Success,2nd Class,60.48
4,056c1b0f-9e46-450f-a688-7b9ad944b86c,186b5901-3a1c-4e6f-aa8f-88f7315af3ab,Success,2nd Class,132.07


# **GDPR & Data Privacy (The "Anonymization" Layer)**

In [22]:
# 1. The Hashing Function
# I used SHA-256, the industry standard

def has_pii(email):
  # Adding a 'salt' (a secret random string) to prevent rainbow table attacks
  salt = 'SNCF_SECRET_SALT_2024!'
  salted_email = email + salt
  return hashlib.sha256(salted_email.encode()).hexdigest()

  # Simulate Raw Data
raw_users = pd.DataFrame({
    'user_id': [1,2,3],
    'name': ['Jean Dupont', 'Marie Curie', 'Pierre Avoine'],
    'email': ['jean.dupont@gmail.com', 'marie.curie@yahoo.fr', 'pierre@sncf.fr']
})

print("🚨 RAW DATA (Internal Only):")
print(raw_users)

🚨 RAW DATA (Internal Only):
   user_id           name                  email
0        1    Jean Dupont  jean.dupont@gmail.com
1        2    Marie Curie   marie.curie@yahoo.fr
2        3  Pierre Avoine         pierre@sncf.fr


In [24]:
# 3. Apply Anonymization
raw_users['email_hash'] = raw_users['email'].apply(has_pii)

In [25]:
# Drop the PII columns
clean_users = raw_users.drop(columns=['email','name'])

In [27]:
print("\n🔒 ANONYMIZED DATA (Ready for Warehouse):")
display(clean_users)


🔒 ANONYMIZED DATA (Ready for Warehouse):


,user_id,email_hash
0,1,f4b77b2ef6a8ae4c63b1a6474b2edc2ddd798da2d24ce4...
1,2,4432c2d4f2ef90e7415f1139a100ec98792d41f3f3f4cd...
2,3,1d94daa78b5c7a6f7c9455bf300be8729b8aeaa8074491...
